This is coded on an ESP8266 motor slider
Very trivial code, no checking or async work

In [118]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.


In [88]:
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.robust/umqtt/robust.py lib/umqtt/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.simple/umqtt/simple.py lib/umqtt/
print(os.listdir("lib/umqtt"))


Sent 204 lines (6411 bytes) to lib/umqtt/simple.py.
['robust.py', 'simple.py']


In [119]:
%sendtofile main.py

import machine, time, network

wifiname = "mqtt-wifi"
wifipassword = "decafbad00"
mqttbroker = "10.3.141.1"

from machine import Pin, ADC
import time

a = ADC(0)
d5 = Pin(14, Pin.OUT)
d6 = Pin(12, Pin.OUT)
d7 = Pin(13, Pin.OUT)
d7.value(0)

time.sleep(0.5)

def gopos(pos):
    il = 0
    for i in range(400):
        d = (a.read() - pos)
        if abs(d) > 8:
            di = 1 if d < 0 else 0
            d5.value(di)
            d6.value(1-di)
            d7.value(1)
            il = i
        else:
            d7.value(0)
            if i-il > 10:
                break
        time.sleep(0.002)
    print(a.read())

gopos(100)   
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)

sta_if.connect(wifiname, wifipassword)
while not sta_if.isconnected():
    time.sleep(0.1)
gopos(900)
print("connected to wifi")


from umqtt.robust import MQTTClient
client = MQTTClient("slider", mqttbroker, 1883)
for i in range(100):
    try:
        client.connect()
        break
    except OSError as e:
        print(e)
    gopos(100+800*(i%2))
print("connected to mqtt")



newpos = None
def sub_cb(topic, msg):
    global newpos
    print((topic, msg))
    try:
        newpos = int(msg)
    except ValueError:
        print("ignore")
        return
    newpos = min(1023, max(0, newpos))

client.set_callback(sub_cb)
client.subscribe(b"slider1/msg")


currpos = a.read()
currt0 = time.ticks_ms()
while 1:
    client.check_msg()
    if newpos is not None:
        gopos(newpos)
        newpos = None
    nextpos = a.read()
    t1 = time.ticks_ms()
    if abs(nextpos - currpos) > 10 and abs(t1 - currt0) > 200:
        print("nextpos", nextpos)
        currpos = nextpos
        currt0 = t1
        client.publish(b"slider1/pos", str(currpos).encode())


Sent 87 lines (1759 bytes) to main.py.


In [90]:
from umqtt.robust import MQTTClient
client = MQTTClient("slider", mqttbroker, 1883)
for i in range(100):
    try:
        client.connect()
        break
    except OSError as e:
        print(e)
    gopos(100+800*(i%2))
print("connected to mqtt")


connected to mqtt


In [95]:
client.publish(b"foo_topic", b"hello")

In [105]:

newpos = None
def sub_cb(topic, msg):
    global newpos
    print((topic, msg))
    try:
        newpos = int(msg)
    except ValueError:
        print("ignore")
        return
    newpos = min(1023, max(0, newpos))

client.set_callback(sub_cb)
client.subscribe(b"slider1/msg")


In [112]:
print(time.ticks_ms())

2638722


In [113]:
currpos = a.read()
currt0 = time.ticks_ms()
while 1:
    client.check_msg()
    if newpos is not None:
        gopos(newpos)
        newpos = None
    nextpos = a.read()
    t1 = time.ticks_ms()
    if abs(nextpos - currpos) > 10 and abs(t1 - currt0) > 200:
        print("nextpos", nextpos)
        currpos = nextpos
        currt0 = t1
        client.publish(b"slider1/pos", str(currpos).encode())


nextpos 626
nextpos 788
nextpos 823
.(b'slider1/msg', b'505')
501
nextpos 493
nextpos 406
bcn_timout,ap_probe_send_start
...............bcn_timout,ap_probe_send_start
.............bcn_timout,ap_probe_send_start
...........(b'slider1/msg', b'505')
485
nextpos 486
nextpos 542
bcn_timout,ap_probe_send_start
......(b'slider1/msg', b'540')
543
(b'slider1/msg', b'317')
340
nextpos 338
(b'slider1/msg', b'338')
328
nextpos 391
(b'slider1/msg', b'391')
391
(b'slider1/msg', b'393')
391
(b'slider1/msg', b'636')
639
nextpos 642
(b'slider1/msg', b'642')
662
nextpos 604
(b'slider1/msg', b'604')
603
.(b'slider1/msg', b'603')
604
(b'slider1/msg', b'316')
340
nextpos 339
(b'slider1/msg', b'339')
320
nextpos 387
(b'slider1/msg', b'387')
388
(b'slider1/msg', b'389')
388
(b'slider1/msg', b'589')
585
nextpos 589
nextpos 684
(b'slider1/msg', b'684')
684
(b'slider1/msg', b'690')
684
(b'slider1/msg', b'283')
276
nextpos 273
nextpos 161
(b'slider1/msg', b'273')
238
nextpos 272
(b'slider1/msg', b'161')
178
next

Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
  File "/lib/umqtt/simple.py", line 204, in check_msg
  File "/lib/umqtt/robust.py", line 42, in wait_msg
  File "/lib/umqtt/robust.py", line 40, in wait_msg
  File "/lib/umqtt/simple.py", line 170, in wait_msg
KeyboardInterrupt: 


In [234]:

from machine import Pin, ADC
import time

a = ADC(0)
d5 = Pin(14, Pin.OUT)
d6 = Pin(12, Pin.OUT)
d7 = Pin(13, Pin.OUT)

def gopos(pos):
    for i in range(100):
        d = (a.read() - pos)
        if abs(d) > 40:
            di = 1 if d < 0 else 0
            d5.value(di)
            d6.value(1-di)
            d7.value(1)
        else:
            d7.value(0)
            break
        time.sleep(0.01)
    print(a.read())

import random
while True:
    pos = random.getrandbits(10)
    gopos(pos)
    time.sleep(0.5)




***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7fa00504b588, open=True>(port='/dev/ttyUSB3', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands